In [1]:
#STUDENT NAME: SHRUTHI
#STUDENT ID: 801218392
#HOMEWORK 3 QUESTION 1-B

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from datetime import datetime

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [3]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [4]:
from torchvision import datasets, transforms
data_path = 'C:/Users/Shruthi/Documents/HWs/RTML/hw2/data'
# Here the training set is downloaded as cifar10

cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [5]:
# The validation set is downloaded as cifar10_val
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [6]:
#The training set is defined as train_loader
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64,
                                           shuffle=False)

In [7]:
#The validation set is defined as val_loader
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

In [8]:
model2 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(16, 8, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),
            nn.Linear(512, 10),
            nn.LogSoftmax(dim=1))

In [9]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):

    correctt = 0
    totalt = 0
    
    for epoch in range (1, n_epochs+1):
        loss_train=0.0
        for imgs, labels in train_loader:
            outputs= model(imgs)
            loss=loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train+=loss.item()
            
        if epoch ==1 or epoch % 10==0:
            now = datetime.now()
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
            print(dt_string, "Epoch: %d, Loss: %f" % (epoch, float(loss)))
            
        with torch.no_grad():
            for imgs, labels in train_loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
            totalt += labels.shape[0]
            correctt += int((predicted == labels).sum())
            acctt = float(correctt/totalt)
    print("Accuracy in Training: ",acctt)

In [10]:
optimizer = optim.SGD(model2.parameters(), lr=1e-2)
loss_fn = nn.NLLLoss()

training_loop(
    n_epochs = 300,
    optimizer = optimizer,
    model = model2,
    loss_fn = loss_fn,
    train_loader = train_loader)

27/03/2022 21:04:26 Epoch: 1, Loss: 2.187056
27/03/2022 21:09:18 Epoch: 10, Loss: 1.683421
27/03/2022 21:14:40 Epoch: 20, Loss: 1.669917
27/03/2022 21:19:54 Epoch: 30, Loss: 1.553355
27/03/2022 21:25:25 Epoch: 40, Loss: 1.414230
27/03/2022 21:30:39 Epoch: 50, Loss: 1.305678
27/03/2022 21:35:46 Epoch: 60, Loss: 1.215417
27/03/2022 21:40:54 Epoch: 70, Loss: 1.173716
27/03/2022 21:46:13 Epoch: 80, Loss: 1.141323
27/03/2022 21:51:43 Epoch: 90, Loss: 1.117276
27/03/2022 21:57:29 Epoch: 100, Loss: 1.079265
27/03/2022 22:03:00 Epoch: 110, Loss: 1.063236
27/03/2022 22:08:21 Epoch: 120, Loss: 1.063122
27/03/2022 22:13:46 Epoch: 130, Loss: 1.058090
27/03/2022 22:19:04 Epoch: 140, Loss: 1.058676
27/03/2022 22:24:41 Epoch: 150, Loss: 1.064309
27/03/2022 22:30:02 Epoch: 160, Loss: 1.055569
27/03/2022 22:34:59 Epoch: 170, Loss: 1.060165
27/03/2022 22:39:59 Epoch: 180, Loss: 1.062684
27/03/2022 22:45:00 Epoch: 190, Loss: 1.062436
27/03/2022 22:50:08 Epoch: 200, Loss: 1.069227
27/03/2022 22:55:22 Epoc

In [11]:
def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correctv = 0
        totalv = 0
        
        with torch.no_grad():
            for imgs, labels in val_loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
            totalv += labels.shape[0]
            correctv += int((predicted == labels).sum())
            acctv = float(correctv/totalv)
    print("Accuracy in Validation: ",acctv)
validate (model2, train_loader, val_loader)

Accuracy in Validation:  0.6875


In [12]:
import torchvision.models as models
from ptflops import get_model_complexity_info

net = model2
macs, params = get_model_complexity_info(net, (3, 32, 32), as_strings=True,
                                           print_per_layer_stat=True, verbose=True)
print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
print('{:<30}  {:<8}'.format('Number of parameters: ', params))

Sequential(
  0.007 M, 100.000% Params, 0.001 GMac, 100.000% MACs, 
  (0): Conv2d(0.0 M, 6.649% Params, 0.0 GMac, 57.509% MACs, 3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(0.0 M, 0.000% Params, 0.0 GMac, 2.054% MACs, )
  (2): MaxPool2d(0.0 M, 0.000% Params, 0.0 GMac, 2.054% MACs, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(0.001 M, 17.216% Params, 0.0 GMac, 37.227% MACs, 16, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU(0.0 M, 0.000% Params, 0.0 GMac, 0.257% MACs, )
  (5): MaxPool2d(0.0 M, 0.000% Params, 0.0 GMac, 0.257% MACs, kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Flatten(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, start_dim=1, end_dim=-1)
  (7): Linear(0.005 M, 76.135% Params, 0.0 GMac, 0.643% MACs, in_features=512, out_features=10, bias=True)
  (8): LogSoftmax(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, dim=1)
)
Computational complexity:       0.0 GMac
Number of parame

In [13]:
numel_list = [p.numel() for p in model2.parameters()]
sum(numel_list), numel_list

(6738, [432, 16, 1152, 8, 5120, 10])